# Autores colombianos por provincias

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores_colombianos_2.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 234 ms, sys: 8.35 ms, total: 242 ms
Wall time: 241 ms


In [3]:
autores_df.shape

(5551, 13)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO,NOMBRE_ENCONTRADO
0,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bahía Solano,Chocó Department,True,Oscar Collazos
1,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bogota,DESCONOCIDO,True,Rafael Pombo
2,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bogota,DESCONOCIDO,True,Rafael Pombo
3,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bogota,DESCONOCIDO,True,Vladdo
4,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Aguadas,DESCONOCIDO,True,Javier Ocampo López


## Ciudades y departamentos

### Ciudades

In [5]:
dtypes = {
    'Ciudad': str,
    'Población': np.int32,
    'Departamento': str,
}

ciudades_df = pd.read_csv(
    'ciudades_colombia.csv',
    sep='\t',
    header=-1,
    names=['Wiki index', 'Ciudad', 'Población', 'Departamento'],
    usecols=['Ciudad', 'Población', 'Departamento'],
    dtype=dtypes,
    thousands='.',
)

In [6]:
ciudades_departamentos_sr = pd.Series(list(ciudades_df['Departamento']), index=ciudades_df['Ciudad'])

In [7]:
ciudades_departamentos_sr.head()

Ciudad
Bogotá             Cundinamarca
Medellín              Antioquia
Cali            Valle del Cauca
Barranquilla          Atlántico
Cartagena               Bolívar
dtype: object

#### Consistencia datasets

In [8]:
for x in autores_df['CIUDAD_AUTOR'].unique():
    if x not in ciudades_departamentos_sr:
        print(x)

Bogota
La Tebaida, Quindío
DESCONOCIDO
Cartagena de Indias
Donmatías
La Ceja del Tambo
Buga
El Socorro
Sincé
Honda, Tolima
El Carmen de Bolívar
Corinto, Cauca
Cartago Municipality
Guapí


In [9]:
cdd_cdd_dict = {}
for c in autores_df['CIUDAD_AUTOR'].unique():
    cdd_cdd_dict[c] = c

In [10]:
cdd_cdd_dict['Bogota'] = 'Bogotá'
cdd_cdd_dict['La Tebaida, Quindío'] = 'La Tebaida'
cdd_cdd_dict['DESCONOCIDO'] = 'DESCONOCIDO'
cdd_cdd_dict['Donmatías'] = 'Don Matías'
cdd_cdd_dict['La Ceja del Tambo'] = 'La Ceja'
cdd_cdd_dict['Buga'] = 'Guadalajara de Buga'
cdd_cdd_dict['El Socorro'] = 'Socorro'
cdd_cdd_dict['Sincé'] = 'San Luis de Sincé'
cdd_cdd_dict['Honda, Tolima'] = 'Honda'
cdd_cdd_dict['El Carmen de Bolívar'] = 'Carmen de Bolivar'
cdd_cdd_dict['Corinto, Cauca'] = 'Corinto'
cdd_cdd_dict['Cartago Municipality'] = 'Cartago'
cdd_cdd_dict['Guapí'] = 'Guapi'

In [11]:
%%time

def process_ciudad(x):
    return cdd_cdd_dict[x['CIUDAD_AUTOR']]

autores_df['CIUDAD_AUTOR'] = autores_df.apply(process_ciudad, axis=1)

CPU times: user 153 ms, sys: 3.97 ms, total: 157 ms
Wall time: 156 ms


### Departamentos

In [12]:
deps1_df = pd.read_csv('deps1.csv', sep='\t', header=-1)

In [13]:
deps1_df.shape

(33, 8)

In [14]:
deps1_df.head()

,0,1,2,3,4,5,6,7
0,0,Capital District,Enrique Peñalosa,CR,Bogotá,"1,587","8,254,722",1538
1,1,Amazonas,Manuel Antonio Carebilla Cuéllar,CR,Leticia,"109,665","80,360",1991
2,2,Antioquia,Luis Pérez,Liberal,Medellín,"63,612","5,750,478",1826
3,3,Arauca,Ricardo Alvarado Bestene,La U,Arauca,"23,818","282,302",1991
4,4,Atlántico,Eduardo I. Verano de la Rosa,Liberal,Barranquilla,"3,388","2,365,663",1910


In [15]:
deps1_df[1] = deps1_df.apply(lambda x: x[1].strip(), axis=1)

In [16]:
deps1_set = set(deps1_df[1])

In [17]:
deps2_df = pd.read_csv('deps2.txt', header=-1)

In [18]:
deps2_df.shape

(33, 1)

In [19]:
deps2_df.head()

,0
0,Antioquia
1,Caldas
2,Bogotá D.C.
3,Bolívar
4,Córdoba


In [20]:
deps2_df[0] = deps2_df.apply(lambda x: x[0].strip(), axis=1)

In [21]:
deps2_set = set(deps2_df[0])

In [22]:
deps_set = deps1_set.union(deps2_set)

In [23]:
deps_set.difference(deps1_set.intersection(deps2_set))

{'Bogotá D.C.', 'Capital District'}

In [24]:
deps_set.remove('Capital District')

In [25]:
len(deps_set)

33

#### Consistencia datasets

In [26]:
dep_dep_dict = {}
for d in deps_set:
    dep_dep_dict[d] = d

In [27]:
dep_dep_dict['Capital District'] = 'Bogotá D.C.'

In [28]:
for x in ciudades_departamentos_sr.index:
    dep = ciudades_departamentos_sr[x]
    if isinstance(dep, str):
        if dep not in deps_set:
            print(x, '-', dep)
    else:
        for d in dep.values:
            if d not in deps_set:
                print(x, '-', d)

San Andrés - Archipiélago de San Andrés
Providencia - Archipiélago de San Andrés
San Andrés - Archipiélago de San Andrés
Providencia - Archipiélago de San Andrés


In [29]:
dep_dep_dict['Archipiélago de San Andrés'] = 'San Andrés y Providencia'

In [30]:
for x in autores_df['DEPARTAMENTO_AUTOR'].unique():
    if x not in deps_set:
        print(x)

Chocó Department
DESCONOCIDO
Quindío Department
Cundinamarca Department
Magdalena Department
Antioquia Department
Nariño Department
Valle del Cauca Department
Atlántico Department
Arauca Department
Santander Department
Cauca Department
Norte de Santander Department
Boyacá Department
Huila Department
Atlántico (Colombia)


In [31]:
dep_dep_dict['Chocó Department'] = 'Chocó'
dep_dep_dict['DESCONOCIDO'] = 'DESCONOCIDO'
dep_dep_dict['Quindío Department'] = 'Quindío'
dep_dep_dict['Cundinamarca Department'] = 'Cundinamarca'
dep_dep_dict['Magdalena Department'] = 'Magdalena'
dep_dep_dict['Antioquia Department'] = 'Antioquia'
dep_dep_dict['Nariño Department'] = 'Nariño'
dep_dep_dict['Valle del Cauca Department'] = 'Valle del Cauca'
dep_dep_dict['Atlántico Department'] = 'Atlántico'
dep_dep_dict['Arauca Department'] = 'Arauca'
dep_dep_dict['Santander Department'] = 'Santander'
dep_dep_dict['Cauca Department'] = 'Cauca'
dep_dep_dict['Norte de Santander Department'] = 'Norte de Santander'
dep_dep_dict['Boyacá Department'] = 'Boyacá'
dep_dep_dict['Huila Department'] = 'Huila'
dep_dep_dict['Atlántico (Colombia)'] = 'Atlántico'

In [32]:
%%time

def process_departamento(x):
    return dep_dep_dict[x['DEPARTAMENTO_AUTOR']]

autores_df['DEPARTAMENTO_AUTOR'] = autores_df.apply(process_departamento, axis=1)

CPU times: user 163 ms, sys: 0 ns, total: 163 ms
Wall time: 160 ms


## Corrigiendo ciudades y departamentos

In [33]:
%%time

inconsistencias = set()
corrections = 0

for i, x in autores_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    departamento = x['DEPARTAMENTO_AUTOR']
    
    if ciudad in ciudades_departamentos_sr:
        deps = ciudades_departamentos_sr[ciudad]
        if departamento == 'DESCONOCIDO':
            if isinstance(deps, str):
                autores_df.loc[i, 'DEPARTAMENTO_AUTOR'] = deps
                corrections += 1
            else:
                if departamento not in deps.values:
                    inconsistencias.add((i, ciudad, departamento, tuple(deps.values)))
        else:
            if isinstance(deps, str):
                if departamento != deps:
                    inconsistencias.add((i, ciudad, departamento, deps))
            else:
                if departamento not in deps.values:
                    inconsistencias.add((i, ciudad, departamento, tuple(deps.values)))

CPU times: user 3.03 s, sys: 1.03 ms, total: 3.03 s
Wall time: 3.01 s


In [34]:
len(inconsistencias)

0

In [35]:
corrections

2019

# Saving results

In [36]:
autores_df.to_csv('autores_colombianos_3.csv', index=False)